# IMPORTS DE LIBRAIRIES

Nous écrivons dans la cellule suivante sur quelle(s) GPU nous souhaitons exécuter le code. Pour spécifier plusieurs GPU, séparer leur id d'une virgule.    
Cependant, le code utilisé ne supporte pas encore le multi GPU.  

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import torch 
print("torch version           : ", torch.__version__)
print("torch cuda version      : ", torch.version.cuda)
print("torch.cuda.is_available : ", torch.cuda.is_available())

In [ ]:
import detectron2
print("detectron2 version : ", detectron2.__version__)

In [ ]:
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.config import get_cfg, get_stack_cell_config
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.data.common import DatasetFromList
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data.datasets import get_dicts
from detectron2.modeling import build_model
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json, cv2, random, glob

Le logger permet d'afficher des informations importantes tout au long de l'exécution des cellules.  

In [ ]:
from detectron2.utils.logger import setup_logger
setup_logger()

In [ ]:
def imBGRshow(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
def imRGBshow(img):
    plt.imshow(img)
    plt.show()

### VISUALISATION DES COURBES D'ENTRAINEMENT
Présent dans ce notebook pour ne pas avoir à chercher la cellule dans le notebook train_custom_z_network.ipynb  
Bien penser à adapter *--logdir* au chemin où les résultats de l'entraînement ont été stockés.

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir "/tmp/TEST/outputs/3D_50_layers"

# REGISTER LES IMAGES
## /!\ CHANGE THE DATA PATH ACCORDINGLY

NB : Les classes sont les suivantes :
- 0 : Cellule intacte et nette   (Intact_Sharp)
- 1 : Cellule intacte et floue   (Intact_Blurry)
- 2 : Cellule explosée et nette  (Broken_Sharp)
- 3 : Cellule explosée et floue  (Broken_Blurry)

Pour seulement considérer les cellules nettes, utiliser :
classes = {'Intact_Sharp':0, 'Broken_Sharp':2}

Pour considérer tous les types de cellules, utiliser :
classes = {'Intact_Sharp':0,'Intact_Blurry':1,'Broken_Sharp':2,'Broken_Blurry':3}  
  
NB: Les entraînements ont été faits avec seulement les cellules nettes

In [ ]:
classes = {'Intact_Sharp':0, 'Broken_Sharp':2}
#classes = {'Intact_Sharp':0,'Intact_Blurry':1,'Broken_Sharp':2,'Broken_Blurry':3}

# DATASET

Les noms de fichiers sont enregistrées au format suivant : TxxPyyFzz avec xx le puceron, yy le cluster, et zz le positionnement de l'image des la pile.
Les images sont au format png. Elles sont de taille 400x300. Chaque image possède un fichier json associé ayant le même nom. Dans celui-ci sont enregistrés des données permettant l'identification de l'image, ainsi que les positions et tailles des boîtes englobantes, les classes, ainsi que les différentes segmentations au format polygon.

## AUGMENTATION DES DONNÉES
L'augmentation des données a été réalisée en amont de l'algorithme. Les données augmentées se trouvent d'ores et déjà dans les dossiers spécifiés au dessus. Seules les augmentations en netteté et en cisaillement n'ont pas été implémentées. En effet, la première cause une netteté de toutes les cellules et ne permettra pas au model d'apprendre la différence entre une cellule nette et une cellule floue. Le cisaillement cause quant à lui une déformation trop importante de la membrane extérieure des bactériocytes, rendant très difficile une différenciation des cellules intactes et des cellules explosées.

35 augmentations différentes ont été implémentées. Le réseau verra donc 36 versions des différentes piles. Les données sont séparées en 5 (voir partie organisation ci-dessous).
- 0 : 15444 images => 1404 piles (depuis 39 piles)
- 1 : 15411 images => 1401 piles (depuis 39 piles)
- 2 : 15015 images => 1365 piles (depuis 39 piles)
- 3 : 15015 images => 1365 piles (depuis 39 piles)
- 4 : 14619 images => 1329 piles (depuis 38 piles)

La partie 4 possède 2 défauts majeurs qu'il convient de prendre en compte : la pile Augmented1T9P20 a une image où la moitié est grise. Il convient donc de supprimer cette pile manuellement. Les piles T10P1Fxx ne possèdent pas 11 images. Celles-ci sont écartées automatiquement par l'algorithme lors du chargement des données. Des piles avec un plus grand nombre d'images que celui attendu seraient chargées jusqu'à leur 11ème image seulement.  
Une fois la pile Augmented1T9P20 supprimée manuellement, il reste 14864 images, dont seulement 14619 seront exploitables comme une pile entière doit être écartée, avec toute ses augmentations. C'est cette pile qui amène le nombre de pile à 38 et non 39 comme les autres parties du dataset.  
  
Des défauts moins importants subsitent : nous remarquons que même si les images sont obtenues à partir du même nombre de piles initiallement, il n'y a pas le même nombre d'image dans chaque partie. Certaines augmentations n'ont pas été réalisées sur certaines piles. 

## ORGANISATION
Le dataset est séparé en 3 jeux de données : 
- 60%    => Entraînement
- 20%    => Validation
- 20%    => Test  
  
Les données doivent être rangées dans la structure suivante de fichiers. La variable data_path définie dans la variable suivante doit indiquer l'emplacement du dossier Cross-val.  
/!\ ATTENTION, ce chemin est à adapter.  
└── Cross-val  
&emsp;&emsp;&emsp;   ├── Xval0  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   ├── Xval1  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   ├── Xval2  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   ├── Xval3  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   └── Xval4  
&emsp;&emsp;&emsp; &emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; &emsp;&emsp;   └── labels  

## VALIDATION CROISEE
Comme son nom l'indique, cette séparation est réalisée afin de pouvoir faire de la validation croisée (cross-validation). Pour des raisons écologiques et de durée d'entraînement, nous n'avons pas tiré profit de cette possibilité, mais il est important de noter qu'elle est facilement implémetable au besoin.  
Un indice indique quelles parties du dataset seront associées avec quel jeu de données (entraînement, validation ou test). Pour réaliser de la validation croisée, il faudra réaliser l'entrainement pour des indices variant de 0 à 4.

In [ ]:
data_path = '/projects/INSA-Image/B01/Data/'
cross_val_idx = 4

In [ ]:
# Modes must have the correct string associated in order to perform the proper operation
mode_test  = 'test'
dataset_name_test  = 'val'      # Pour correspondre à faire l'inférence et pas le training selon la confiuration . La partie du dataset loadée est bien définie par le mode_tes

In [ ]:
# Register the datasets
DatasetCatalog.register(dataset_name_test,  lambda: get_dicts(data_path, mode_test,  cross_val_idx, classes, dataset_name_test))
# Set the evaluator to the coco evaluator
MetadataCatalog.get('val').set(evaluator_type="coco")

# CHARGEMENT DE LA CONFIGURATION EXISTANTE

Les meilleurs poids sont:
- BACKBONE 2D, 18 LAYERS (10 000 iterations) : model_0000499.pth
- BACKBONE 2D, 50 LAYERS (11 000 iterations) : model_0009999.pth
- BACKBONE 3D, 18 LAYERS (10 000 iterations) : model_0000499.pth
- BACKBONE 3D, 50 LAYERS (30 000 iterations) : model_0000499.pth

In [ ]:
# Chemin jusqu'aux fichiers contenant les poids du réseau et sa configuration
path_to_weights_and_config = "/tmp/TEST/outputs/test/"
# Fichier de poids à utiliser
weight_file = "model_final.pth"
# Chemin dans lequel sauvegarder les images segmentées par l'algorithme et la vérité terrain
output_directory = "/tmp/TEST/outputs/test/"

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(os.path.join(path_to_weights_and_config, "config.yaml"))    # Chargement de la configuration existante

# Adaptation pour l'inférence
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.SOLVER.IMS_PER_BATCH = 1                    # Attention, doit être réadapté, surtout si l'entraînement était en multigpu
cfg.MODEL.WEIGHTS = os.path.join(path_to_weights_and_config, weight_file)
cfg.REFERENCE_WORLD_SIZE = 1                    # Doit être au nombre de gpu * nombre de machine, pour le jupyter notebook, à 1 comme fonctionne à 1 GPU sur 1 machine

# VISUALISATION

In [ ]:
# Construction du predicteur
predictor = DefaultPredictor(cfg)

In [ ]:
# On récupère le metadata du dataset, une liste de ses dictionnaires, et enfin le dataset lui-même, en format torch, qui est nécessaire pour faire la prédiction sur la pile
test_metadata = MetadataCatalog.get(dataset_name_test)
test_dataset_dicts = DatasetCatalog.get(dataset_name_test)
test_dataset = DatasetFromList(test_dataset_dicts, cfg.DATALOADER.IS_STACK, cfg.INPUT.STACK_SIZE, serialize=False)
# Attention : si le dataset est serialized alors on ne pourra pas accéder correctement à la liste de tous les dictionnaires de la pile.
# La serialisation (en pickle ici) est cependant très intéressante pour stocker et transmettre des données, ce qui est utile dans d'autres parties du programme

In [ ]:
# Visualisation de N piles choisies au hasard dans le dataset
N = 1
stack = [None] * cfg.INPUT.STACK_SIZE
out   = [None] * cfg.INPUT.STACK_SIZE
for data in random.sample(test_dataset._lst, N):

    # Construction de la pile d'image à partir du dataset
    for z in range(cfg.INPUT.STACK_SIZE):
        stack[z] = cv2.imread(data[z]["file_name"])

    # Prédiction sur la pile
    outputs = predictor(stack)

    # Affichage
    for z in range(cfg.INPUT.STACK_SIZE):
        print(z)

        visualizer_GT = Visualizer(stack[z][:, :, ::-1], metadata=test_metadata, scale=1)       # Construit le visualiser
        out_GT = visualizer_GT.draw_dataset_dict(data[z])                                       # Dessine les annotations sur l'image
        print("Ground Truth")
        imRGBshow(out_GT.get_image())                                                           # Display l'image
        cv2.imwrite(os.path.join(output_directory,  f"GTimage_{z}.jpg"), cv2.cvtColor(out_GT.get_image(), cv2.COLOR_RGB2BGR))    # Enregistre l'image

        visualizer = Visualizer(stack[z][:, :, ::-1], metadata=test_metadata, scale=1)
        out[z] = visualizer.draw_instance_predictions(outputs[z]["instances"].to("cpu"))
        print("Predicted")
        imRGBshow(out[z].get_image())
        cv2.imwrite(os.path.join(output_directory,  f"DTimage_{z}.jpg"), cv2.cvtColor(out[z].get_image(), cv2.COLOR_RGB2BGR))